<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Sntence_entailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**work in progress**

# Data Exploration

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!unzip train.zip

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train_filtered  = train.filter(['premise','hypothesis','label'])

In [ ]:
train_filtered.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [ ]:
#get cell value
train.iloc[1]['premise']
train.iloc[1]['hypothesis']

In [ ]:
#A way to split a dataset in NLP 
dataset = nlp.Dataset.from_pandas(train_filtered)
ds_splitted = dataset.train_test_split(test_size=0.1)

In [ ]:
ds_splitted['train']['premise'][1]

'But in all probability the girl will have entirely forgotten the intervening period, and will take up life where she left off at the sinking of the Lusitania."'

# Model

In [3]:
%%capture
!pip install pytorch_lightning
!pip install transformers
!pip install nlp

In [4]:
import torch as th
import pytorch_lightning as pl
import nlp
import transformers

In [5]:
from transformers import BertTokenizer

In [ ]:
#example of tokenizing 2 sentences
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = ds_splitted['train']['premise'][1]
sequence_b = ds_splitted['train']['hypothesis'][1]
encoded_dict = tokenizer(sequence_a, sequence_b,                     
                         max_length=128, 
                         truncation = True,
                         pad_to_max_length=True,
                         return_tensors = 'pt')
#decoded = tokenizer.decode(encoded_dict["input_ids"])
print(type(encoded_dict))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:
#example of feeding input into the Base Bert Model
model = transformers.BertModel.from_pretrained('bert-base-uncased')
outputs = model(**encoded_dict)

In [ ]:
# Shape of pooled output
outputs[1].shape

torch.Size([1, 768])

**Pytorch Model using Torch Lightning**

In [ ]:
# using basic BERT with dense layer on top
LOSS = []
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        #self.brt = transformers.BertForSequenceClassification.from_pretrained('bert-base-cased')
        self.brt = transformers.BertModel.from_pretrained('bert-base-cased')
        self.layer1 = th.nn.Linear(768,3) # one dense layer on top of base Bert
        #self.relu = th.nn.ReLU()
        #self.layer2 = th.nn.Linear(32,3)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

        def _tokenize(x):
            print('in tokenize')
            return tokenizer(
                    x['premise'],
                    x['hypothesis'],
                    max_length=64, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            df = pd.read_csv('train.csv', nrows=200)
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = nlp.Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)

            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            return ds_flt['train'], ds_flt['test']

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks, token_type_ids):
        logits = self.brt(input_ids, masks, token_type_ids)
        out = self.layer1(logits[1]) # Using pooled output , model with a dense layer
        #out = logits[1] # using sequnce pretrained
        #out = self.layer1(logits[0][:,0,:]) # Using first token of the sentences sequence
        #out = self.relu(out)
        #out = self.layer2(out)
        return out

    def training_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label']).mean()
        LOSS.append(loss.tolist())
        training_loss = {'train_loss': loss}
        return {'loss': loss, 'log': training_loss} 

    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label'])
        acc = (logits.argmax(-1) == batch['label']).float() # argmax(1) or argmax(dim=1) produces the same result
        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        print('in validation epoc end: out: {}'.format(out)) 
        return {**out, 'log': out}

    def train_dataloader(self):
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=32,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=8,
                drop_last=False,
                shuffle=False,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [ ]:
#!rm -rf ./logs/
def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=4,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs/', name='entail'),
    )
    trainer.fit(model)

!rm -rf ./logs/

start_time = time.time()
model_fit()
execution_time = time.time() - start_time

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



in tokenize


in tokenize

in tokenize


in tokenize



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name   | Type             | Params
--------------------------------------------
0 | brt    | BertModel        | 109 M 
1 | layer1 | Linear           | 2 K   
2 | loss   | CrossEntropyLoss | 0     


...

in validation epoc end: out: {'val_loss': tensor(1.1050, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}


in validation epoc end: out: {'val_loss': tensor(1.1235, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}


in validation epoc end: out: {'val_loss': tensor(1.1538, device='cuda:0'), 'val_acc': tensor(0.3000, device='cuda:0')}


Saving latest checkpoint..


in validation epoc end: out: {'val_loss': tensor(1.2180, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}



In [ ]:
print('training execution time - {}'.format(execution_time))

training execution time - 41.522226333618164


In [ ]:
len(LOSS)

20

In [ ]:
#%reload_ext tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

**Pretrained Model**

In [22]:
# using basic BERT for sequence classification
LOSS = []
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.brt = transformers.BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels = 3)

        #self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')

        def _tokenize(x):
            print('in tokenize')
            return tokenizer(
                    x['premise'],
                    x['hypothesis'],
                    max_length=64, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            df = pd.read_csv('train.csv', nrows=50)
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = nlp.Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)

            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            return ds_flt['train'], ds_flt['test']

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks, token_type_ids, labels):
        print('in forward')
        out = self.brt(input_ids, masks, token_type_ids, labels = labels)
        print('forward brt output - {}'.format(out))
        return out

    def training_step(self, batch, batch_idx):
        res = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'], batch['label'])
        LOSS.append(res[0].tolist())
        training_loss = {'train_loss': res[0]}
        return {'loss': res[0], 'log': training_loss} 

    def validation_step(self, batch, batch_idx):
        print('in validation step')
        res = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'], batch['label'])
        print('forward output - {}'.format(res))
        loss = res[0]
        acc = (res[1].argmax(-1) == batch['label']).float() # argmax(1) or argmax(dim=1) produces the same result
        return {'loss': loss, 'acc': acc}

    '''def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        print('in validation epoc end: out: {}'.format(out)) 
        return {**out, 'log': out}'''

    def train_dataloader(self):
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=1,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=1,
                drop_last=False,
                shuffle=False,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [23]:
def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=4,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs_bert/', name='pretrained'),
    )
    trainer.fit(model)

!rm -rf ./logs_bert/

start_time = time.time()
model_fit()
execution_time = time.time() - start_time

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b


in tokenize


in tokenize

in tokenize


in tokenize



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name | Type                          | Params
-------------------------------------------------------
0 | brt  | BertForSequenceClassification | 108 M 


...

in validation step
in forward
forward brt output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
forward output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
forward output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
forward output - (tensor(17.7745, device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(0., device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], device='cuda:0'))
forward output - (tensor(0., device='cuda:0'), tensor([[ 39.8285, -62.2798,  22.0540]], devi

in validation step
in forward
forward brt output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
forward output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
forward output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
forward output - (tensor(7.4405, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(0.0006, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], device='cuda:0'))
forward output - (tensor(0.0006, device='cuda:0'), tensor([[ 13.9801, -20.9371,   6.5402]], de

in validation step
in forward
forward brt output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
forward output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
forward output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
forward output - (tensor(0., device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
in validation step
in forward
forward brt output - (tensor(69.7804, device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
forward output - (tensor(69.7804, device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
in val

Saving latest checkpoint..


forward brt output - (tensor(169.6014, device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))
forward output - (tensor(169.6014, device='cuda:0'), tensor([[  9.8701, -89.9509,  79.6505]], device='cuda:0'))



In [24]:
LOSS

[0.9005375504493713,
 0.005486905109137297,
 4.649054244509898e-05,
 4.768370445162873e-07,
 17.779802322387695,
 0.6644642949104309,
 0.5502371191978455,
 11.17055892944336,
 5.837680339813232,
 1.2397689715726301e-05,
 26.787460327148438,
 26.732666015625,
 13.98500919342041,
 10.054161071777344,
 7.331775665283203,
 0.00011300401820335537,
 52.487213134765625,
 0.0,
 37.9504508972168,
 7.390948667307384e-06,
 0.38571158051490784,
 0.5918413996696472,
 0.4421452283859253,
 18.410167694091797,
 5.860188961029053,
 21.970273971557617,
 0.0,
 0.0,
 0.0,
 65.22087860107422,
 44.325775146484375,
 15.814786911010742,
 0.0,
 44.66754913330078,
 80.01502990722656,
 30.0107479095459,
 6.288298606872559,
 4.0531076592742465e-06,
 0.0,
 64.62715911865234,
 0.0,
 70.06698608398438,
 8.106198947643861e-06,
 61.50950241088867,
 41.534976959228516,
 15.151175498962402,
 13.669014930725098,
 5.7457780838012695,
 68.36756896972656,
 20.86844825744629,
 0.0,
 0.04573346674442291,
 0.031767722219228745

# Sandbox
*various code snippets*

In [ ]:
# TrainResult object if I ever need it...
loss = th.Tensor([0.0584,-0.1041,0.1367])
result = pl.TrainResult()
result.log('train_loss', loss)
print(result)

{'train_loss': tensor([ 0.0584, -0.1041,  0.1367])}


In [ ]:
#Example of Softmax calculation in Pytorch
inp = th.Tensor([0.0584,-0.1041,0.1367]); inp
probs = th.nn.Softmax(dim=-1); probs
res = probs(inp); res

tensor([0.3411, 0.2900, 0.3689])

In [ ]:
# Using tokenizer example
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs[0]